# Import Libraries

In [0]:
import bs4
import requests
import re
import string
import pandas as pd
import numpy as np

# Scraping function

In [0]:
actor_name=[]
actor_image=[]
bio=[]
test_list = list(string.ascii_uppercase) # list of all uppercase English alphabets
def fun(url):
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.content)
    for i in test_list:
        try:
            heading = soup.find(id=i) # find header tag
            teams = heading.find_next('ul') # find unordered list under the header tag
            for team in teams:
                x=team.string # name of celebrity
                if(x!='\n'):
                    k='https://en.wikipedia.org/wiki/'+x.replace(' ','_') # URL of personal page
                    bio.append(k)
                    html = requests.get(k)
                    bs = bs4.BeautifulSoup(html.text,'html.parser')
                    images = bs.find_all('img', {'src':re.compile('.jpg')}) # extract image links from celebrity's personal page
                    if(images!=[]):
                        actor_name.append(x) # append name
                        actor_image.append(images[0]['src']+'\n') # append first image
                    else:
                        actor_name.append(x)
                        actor_image.append('')
        except Exception:
            pass
    return actor_name,actor_image

# Scrape data from first URL

In [0]:
an,ai=fun('https://en.wikipedia.org/wiki/List_of_Bollywood_actors')

# Scrape data from some outbound URLs

In [0]:
l=[]
r = requests.get('https://en.wikipedia.org/wiki/List_of_Bollywood_actors')
soup = bs4.BeautifulSoup(r.content)
heading = soup.find(id='See_also') # find "See also" header tag
teams = heading.find_next('ul') # find unordered list under "See also" header tag
for team in teams:
    x=team.string
    if(x!='\n'):
        l.append('https://en.wikipedia.org/wiki/'+x.replace(' ','_')) #store all URLs under "See also" header tag

In [5]:
l

['https://en.wikipedia.org/wiki/List_of_Indian_film_actors',
 'https://en.wikipedia.org/wiki/List_of_Indian_film_actresses',
 'https://en.wikipedia.org/wiki/List_of_Bollywood_actresses',
 'https://en.wikipedia.org/wiki/List_of_Indian_television_actors',
 'https://en.wikipedia.org/wiki/Lists_of_actors']

In [0]:
for i in l[:3]: # scrape data from first three URLs (others are irrelevant)
    an,ai=fun(i)

# Output DataFrame

In [8]:
df=pd.DataFrame(zip(an,ai,bio),columns=['Celebrity','Image','Bio'])
df=df.replace('', np.nan) # replace blank values with NaN
df['Celebrity']=df['Celebrity'].str.casefold() # lowercase string in "Actor" column
df.sort_values("Celebrity", inplace = True)  # sort DataFrame based on "Actor" column
df.drop_duplicates(subset ="Celebrity",inplace = True) # drop a row with duplicate "Actor" entry
df                

,Celebrity,Image,Bio
37,a. k. hangal,//upload.wikimedia.org/wikipedia/commons/thumb...,https://en.wikipedia.org/wiki/A._K._Hangal
310,aadhi,//upload.wikimedia.org/wikipedia/en/thumb/e/ee...,https://en.wikipedia.org/wiki/Aadhi
311,aadi pudipeddi,NaN,https://en.wikipedia.org/wiki/Aadi_Pudipeddi
51,aamir bashir,//upload.wikimedia.org/wikipedia/commons/thumb...,https://en.wikipedia.org/wiki/Aamir_Bashir
0,aamir khan,//upload.wikimedia.org/wikipedia/commons/thumb...,https://en.wikipedia.org/wiki/Aamir_Khan
...,...,...,...
1033,zeishan quadri,//upload.wikimedia.org/wikipedia/commons/thumb...,https://en.wikipedia.org/wiki/Zeishan_Quadri
1623,zoya khan,//upload.wikimedia.org/wikipedia/commons/thumb...,https://en.wikipedia.org/wiki/Zoya_Khan
308,zubeen garg,//upload.wikimedia.org/wikipedia/commons/thumb...,https://en.wikipedia.org/wiki/Zubeen_Garg
1624,zubeida,//upload.wikimedia.org/wikipedia/commons/9/97/...,https://en.wikipedia.org/wiki/Zubeida
